In [ ]:
C2 = 20
f = 1000 # frequency [Hz]
c = 346 # speed of sound in air [m/s]
wavelength = c/f

e = 1 # distance between the screens in the direction of the source and receiver [m]
d_ss = 1 # perpendicular distance from source to plane of screen [m]
d_sr = 1 # perpendicular distance from plane of screen to receiver [m]